In [ ]:
from autogluon import TabularPrediction as task
import time

In [ ]:
dataset = 'dataset'
!kaggle competitions download -p {dataset} -q otto-group-product-classification-challenge
!unzip -d {dataset} {dataset}/otto-group-product-classification-challenge.zip
!rm {dataset}/otto-group-product-classification-challenge.zip

In [ ]:
train_data = task.Dataset(file_path=f'{dataset}/train.csv').drop('id', axis=1)
train_data.head()

In [ ]:
label_column = 'target' # specifies which column do we want to predict
savedir = 'otto_models/' # where to save trained models

In [ ]:
predictor = task.fit(train_data=train_data, 
                     label=label_column, 
                     output_directory=savedir, 
                     eval_metric='log_loss', 
                     auto_stack=True,
                     verbosity=2,
                     visualizer='tensorboard')

In [ ]:
results = predictor.fit_summary() # display detailed summary of fit() process

In [ ]:
# predictor = task.load('otto_models')

In [ ]:
dataset = 'dataset'
test_data_full = task.Dataset(file_path=f'{dataset}/test.csv')
test_data = test_data_full.drop('id', axis=1)

In [ ]:
%%time
pred_probablities = predictor.predict_proba(test_data, as_pandas=True)

In [ ]:
pred_probablities.insert(loc=0, column='id', value=test_data_full['id'])

In [ ]:
submission_name = f'submission-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}.csv'
pred_probablities.to_csv(submission_name,index=False)
!kaggle competitions submit otto-group-product-classification-challenge -f {submission_name} -m "autogluon {submission_name}"